## DR.DECR (Dense Retrieval with Distillation-Enhanced Cross-Lingual Representation) Student/Teacher Training
In this notebook, we show how to train a model using Knowledge Distillation (Student/Teacher) to improve performance on Cross-lingual retrieval, as desribed in `Learning Cross-Lingual IR from an English Retriever`, https://arxiv.org/abs/2112.08185.

After training the model, we index the data and run search.  


In orded to run (almost) instantaneously, we use trivial sizes of training data and collection to search.

## Dependencies
If not already done, please make sure to install PrimeQA with notebooks extras before getting started.

In [1]:
# If you want CUDA 11 uncomment and run this (for CUDA 10 or CPU you can ignore this line).
#! pip install 'torch~=1.11.0' --extra-index-url https://download.pytorch.org/whl/cu113

# Uncomment to install PrimeQA from source (pypi package pending).
# The path should be the project root (e.g. '.' below).
#! pip install .[notebooks]

## Configuration
First, we need to include the required modules.

In [2]:
import os
import tempfile
 
from primeqa.ir.dense.colbert_top.colbert.utils.utils import create_directory, print_message
from primeqa.ir.dense.colbert_top.colbert.infra import Run, RunConfig
from primeqa.ir.dense.colbert_top.colbert.infra.config import ColBERTConfig
from primeqa.ir.dense.colbert_top.colbert.training.training import train
from primeqa.ir.dense.colbert_top.colbert.indexing.collection_indexer import encode
from primeqa.ir.dense.colbert_top.colbert.searcher import Searcher

## Training
We will train a ColBERT model using a TSV file containing [query, positive document, negative document] triples. We use the XOR-TyDi dataset, as described here: https://nlp.cs.washington.edu/xorqa/

The path in `test_files_location` below points to the location of files used by the notebook, by default it points to the files used by CI testing.

In [3]:
test_files_location = '../../../tests/resources/ir_dense'
model_type = 'xlm-roberta-base'
with tempfile.TemporaryDirectory() as working_dir:
    output_dir=os.path.join(working_dir, 'output_dir')
text_triples_fn = os.path.join(test_files_location, "xorqa.train_ir_negs_5_poss_1_001pct_at_0pct.tsv")

Here is an example of a training file record:

In [4]:
import pandas as pd
from IPython.display import display, HTML
data = pd.read_csv(text_triples_fn, sep='\t', nrows=1, header=None)
display(HTML(data.to_html()))

,0,1,2
0,중국에서 가장 오랜기간 왕위를 유지한 인물은 누구인가?,"Kangxi Emperor The Kangxi Emperor's reign of 61 years makes him the longest-reigning emperor in Chinese history (although his grandson, the Qianlong Emperor, had the longest period of ""de facto"" power) and one of the longest-reigning rulers in the world. However, since he ascended the throne at the age of seven, actual power was held for six years by four regents and his grandmother, the Grand Empress Dowager Xiaozhuang.","Chiddy Bang new songs from the duo and in November 2009 debuted the group's first free mixtape entitled ""The Swelly Express"". On 28 April 2011 during the first-ever MTV O Music Awards, Anamege broke the Guinness World Record for Longest Freestyle Rap and Longest Marathon Rapping Record by freestyling for 9 hours, 18 minutes, and 22 seconds, stealing the throne from rapper M-Eighty, who originally broke the record in 2009 by rapping for 9 hours, 15 minutes and 15 seconds. Anamege had also beat Canadian rapper D.O. for Longest Marathon Rapping session, the previous record being for 8 hours and 45 minutes."


In [5]:
args_dict = {
                'root': output_dir,
                'experiment': 'test_training_student',
                'triples': text_triples_fn,
                'similarity': 'l2',
                'model_type': model_type,
                'maxsteps': 3,
                'bsize': 1,
                'accumsteps': 1,
                'amp': True,
                'epochs': 1,
                'rank': 0,
                'nranks': 1
            }

Next we train the the student starting-point model, and save it's location in the `student_model_fn`variable

In [6]:
with Run().context(RunConfig(root=args_dict['root'], experiment=args_dict['experiment'], nranks=args_dict['nranks'], amp=args_dict['amp'])):
    colBERTConfig = ColBERTConfig(**args_dict)   
    student_model_fn = train(colBERTConfig, text_triples_fn, None, None)

{
    "nprobe": 2,
    "ncandidates": 8192,
    "index_path": null,
    "nbits": 1,
    "kmeans_niters": 20,
    "num_partitions_max": 10000000,
    "similarity": "l2",
    "bsize": 1,
    "accumsteps": 1,
    "lr": 3e-6,
    "maxsteps": 3,
    "save_every": null,
    "resume": false,
    "resume_optimizer": false,
    "warmup": null,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": false,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "shuffle_every_epoch": false,
    "save_steps": 2000,
    "save_epochs": -1,
    "epochs": 1,
    "input_arguments": {},
    "model_type": "xlm-roberta-base",
    "init_from_lm": null,
    "local_models_repository": null,
    "ranks_fn": null,
    "topK": 100,
    "student_teacher_temperature": 1.0,
    "student_teacher_top_loss_weight": 0.5,
    "teacher_model_type": "xlm-roberta-base",
    "teacher_doc_maxlen": 180,
    "distill_query_passage_separately": false,
    "query_only": 

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing HF_ColBERT_XLMR: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing HF_ColBERT_XLMR from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HF_ColBERT_XLMR from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HF_ColBERT_XLMR were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['bert.encoder.layer.10.attention.output.dense.weight', 'bert.encoder.layer.10.intermediate.dense.weight', 'bert.encoder.layer.8.attention.self.key.weight', '

[May 12, 15:40:41] maxsteps: 3
[May 12, 15:40:41] 1 epochs of 5 examples
[May 12, 15:40:41] batch size: 1
[May 12, 15:40:41] maxsteps set to 3
[May 12, 15:40:41] start batch idx: 0
[May 12, 15:40:41] #> XMLR QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
[May 12, 15:40:41] #> Input: $ 중국에서 가장 오랜기간 왕위를 유지한 인물은 누구인가?, 		 True, 		 None
[May 12, 15:40:41] #> Output IDs: torch.Size([32]), tensor([     0,   9748,  24120,   1180,  13968, 211059,  83639,  76826,  78363,
         57104,    993, 161732,    697, 116932, 114150,     32,      2,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1])
[May 12, 15:40:41] #> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
[May 12, 15:40:41] #> XLMR DocTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
[May 12, 15:40:41] #> Input: $ "Kangxi

/u/franzm/packages/minconda3/envs/ColBERT_primeqa_7/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/u/franzm/packages/minconda3/envs/ColBERT_primeqa_7/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
/u/franzm/packages/minconda3/envs/ColBERT_primeqa_7/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[May 12, 15:40:42] #>>>> before linear query ==
[May 12, 15:40:42] #>>>>> Q: torch.Size([32, 768]), tensor([[ 0.1269,  0.0868,  0.0588,  ..., -0.0978,  0.0423,  0.0024],
        [ 0.0237,  0.0695,  0.0303,  ..., -0.0671, -0.0818, -0.0653],
        [ 0.0626,  0.0780,  0.0324,  ...,  0.0213, -0.0027,  0.1598],
        ...,
        [ 0.1314,  0.0969,  0.0671,  ..., -0.1059,  0.0561, -0.0338],
        [ 0.0398,  0.0168,  0.0476,  ..., -0.0210, -0.0167, -0.0480],
        [ 0.0871,  0.0832,  0.0208,  ..., -0.0546,  0.0003,  0.0225]],
       grad_fn=<SelectBackward0>)
[May 12, 15:40:42] #>>>>> self.linear query : Parameter containing:
tensor([[-1.2613e-02, -3.6824e-03, -1.0773e-02,  ..., -1.3896e-02,
          1.2244e-02, -8.2326e-03],
        [-3.4988e-03,  3.9623e-03, -2.4905e-02,  ..., -2.6214e-02,
          4.0837e-03, -8.0812e-03],
        [-1.1158e-02, -2.1660e-05,  1.5019e-02,  ..., -1.1310e-02,
         -9.3691e-03, -2.2202e-02],
        ...,
        [-1.0295e-02,  3.2577e-02,  3.3341

Next we train the the teacher model, and save its location in the `teacher_model_fn`variable.

When training this model we use data with the same passages as for the student model, but with English translations of the queries.

In [7]:
text_triples_en_fn = os.path.join(test_files_location, "xorqa.train_ir_negs_5_poss_1_001pct_at_0pct_en.tsv")
args_dict = {
                'root': output_dir,
                'experiment': 'test_training_teacher',
                'triples': text_triples_en_fn,
                'similarity': 'l2',
                'model_type': model_type,
                'maxsteps': 3,
                'bsize': 1,
                'accumsteps': 1,
                'amp': True,
                'epochs': 1,
                'rank': 0,
                'nranks': 1
            }

In [8]:
with Run().context(RunConfig(root=args_dict['root'], experiment=args_dict['experiment'], nranks=args_dict['nranks'], amp=args_dict['amp'])):
    colBERTConfig = ColBERTConfig(**args_dict)   
    teacher_model_fn = train(colBERTConfig, text_triples_en_fn, None, None)

{
    "nprobe": 2,
    "ncandidates": 8192,
    "index_path": null,
    "nbits": 1,
    "kmeans_niters": 20,
    "num_partitions_max": 10000000,
    "similarity": "l2",
    "bsize": 1,
    "accumsteps": 1,
    "lr": 3e-6,
    "maxsteps": 3,
    "save_every": null,
    "resume": false,
    "resume_optimizer": false,
    "warmup": null,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": false,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "shuffle_every_epoch": false,
    "save_steps": 2000,
    "save_epochs": -1,
    "epochs": 1,
    "input_arguments": {},
    "model_type": "xlm-roberta-base",
    "init_from_lm": null,
    "local_models_repository": null,
    "ranks_fn": null,
    "topK": 100,
    "student_teacher_temperature": 1.0,
    "student_teacher_top_loss_weight": 0.5,
    "teacher_model_type": "xlm-roberta-base",
    "teacher_doc_maxlen": 180,
    "distill_query_passage_separately": false,
    "query_only": 

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing HF_ColBERT_XLMR: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing HF_ColBERT_XLMR from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HF_ColBERT_XLMR from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HF_ColBERT_XLMR were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['bert.encoder.layer.10.attention.output.dense.weight', 'bert.encoder.layer.10.intermediate.dense.weight', 'bert.encoder.layer.8.attention.self.key.weight', '

[May 12, 15:41:29] maxsteps: 3
[May 12, 15:41:29] 1 epochs of 5 examples
[May 12, 15:41:29] batch size: 1
[May 12, 15:41:29] maxsteps set to 3
[May 12, 15:41:29] start batch idx: 0
[May 12, 15:41:29] #> XMLR QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
[May 12, 15:41:29] #> Input: $ Who maintained the throne for the longest time in China?, 		 True, 		 None
[May 12, 15:41:29] #> Output IDs: torch.Size([32]), tensor([    0,  9748, 40469, 76104,   297,    70,     6, 42294,    86,   100,
           70,  4989,   525,  1733,    23,  9098,    32,     2,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1])
[May 12, 15:41:29] #> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
[May 12, 15:41:29] #> XLMR DocTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
[May 12, 15:41:29] #> Input: $ "Kangxi Emper

/u/franzm/packages/minconda3/envs/ColBERT_primeqa_7/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/u/franzm/packages/minconda3/envs/ColBERT_primeqa_7/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
/u/franzm/packages/minconda3/envs/ColBERT_primeqa_7/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[May 12, 15:41:29] #>>>> before linear query ==
[May 12, 15:41:29] #>>>>> Q: torch.Size([32, 768]), tensor([[ 0.1266,  0.1302,  0.0628,  ..., -0.1106,  0.0519, -0.0265],
        [-0.0445,  0.0772,  0.0160,  ..., -0.0904, -0.0560, -0.0523],
        [-0.0128,  0.0967, -0.0064,  ..., -0.0320, -0.0303,  0.1300],
        ...,
        [ 0.1200,  0.1188, -0.0589,  ..., -0.2018, -0.0487, -0.0523],
        [ 0.0734,  0.0938, -0.0106,  ..., -0.1310, -0.0079,  0.0202],
        [ 0.1056,  0.1080, -0.0078,  ..., -0.0156, -0.0259,  0.0222]],
       grad_fn=<SelectBackward0>)
[May 12, 15:41:29] #>>>>> self.linear query : Parameter containing:
tensor([[-1.2613e-02, -3.6824e-03, -1.0773e-02,  ..., -1.3896e-02,
          1.2244e-02, -8.2326e-03],
        [-3.4988e-03,  3.9623e-03, -2.4905e-02,  ..., -2.6214e-02,
          4.0837e-03, -8.0812e-03],
        [-1.1158e-02, -2.1660e-05,  1.5019e-02,  ..., -1.1310e-02,
         -9.3691e-03, -2.2202e-02],
        ...,
        [-1.0295e-02,  3.2577e-02,  3.3341

In the following two Knowledge Distillation (KD) steps, we will train the student model using 
1. parallel data contatining a) English and non-English passages for the student model, and b) English passages for the teacher model, 
2. parallel data containg a) non-English queries and English passages for the student model, and b) English queries and English passages for the student model.

First, we run Knowledge Distillation where the student learns teacher's token representations using the parallel English and non-English passage data from the following two files: 

a) English and non-English passages

b) English only passages:

In [9]:
parallel_non_en_fn = os.path.join(test_files_location, "7lan_notrim_triple_2ep.other.clean.h5")
parallel_en_fn = os.path.join(test_files_location, "7lan_notrim_triple_2ep.en.clean.h5")

Here is an example item from the mixed-language file:

In [10]:
data = pd.read_csv(parallel_non_en_fn, sep='\t', nrows=1, header=None)
display(HTML(data.to_html()))

,0,1,2
0,সে একটা জরুরি পুনর্বাসনের কাজ করছিল,She was right in the middle of an important piece of restoration .,She was right in the middle of an important piece of restoration .


Here are the parameters and command for the first Knowledge Distillation (KD) stage:

In [11]:
parallel_non_en_fn = os.path.join(test_files_location, "7lan_notrim_triple_2ep.other.clean.h5")
parallel_en_fn = os.path.join(test_files_location, "7lan_notrim_triple_2ep.en.clean.h5")

args_dict = {
    'root': output_dir,
    'experiment': 'test_training',
    'similarity': 'l2',
    'model_type': model_type,
    'checkpoint': student_model_fn + '-LAST.dnn',
    'distill_query_passage_separately': True,
    'teacher_model_type': model_type,
    'teacher_checkpoint': teacher_model_fn + '-LAST.dnn',
    'triples': parallel_non_en_fn,
    'teacher_triples': parallel_en_fn,
    'maxsteps': 3,
    'bsize': 1,
    'accumsteps': 1,
    'amp': True,
    'epochs': 1,
    'rank': 0,
    'nranks': 1
}

with Run().context(RunConfig(root=args_dict['root'], experiment=args_dict['experiment'], nranks=args_dict['nranks'], amp=args_dict['amp'])):
    colBERTConfig = ColBERTConfig(**args_dict)
    stage_1_student_model_fn = train(colBERTConfig, parallel_non_en_fn, None, None)


{
    "nprobe": 2,
    "ncandidates": 8192,
    "index_path": null,
    "nbits": 1,
    "kmeans_niters": 20,
    "num_partitions_max": 10000000,
    "similarity": "l2",
    "bsize": 1,
    "accumsteps": 1,
    "lr": 3e-6,
    "maxsteps": 3,
    "save_every": null,
    "resume": false,
    "resume_optimizer": false,
    "warmup": null,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": false,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "shuffle_every_epoch": false,
    "save_steps": 2000,
    "save_epochs": -1,
    "epochs": 1,
    "input_arguments": {},
    "model_type": "xlm-roberta-base",
    "init_from_lm": null,
    "local_models_repository": null,
    "ranks_fn": null,
    "topK": 100,
    "student_teacher_temperature": 1.0,
    "student_teacher_top_loss_weight": 0.5,
    "teacher_model_type": "xlm-roberta-base",
    "teacher_doc_maxlen": 180,
    "distill_query_passage_separately": true,
    "query_only": f

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing HF_ColBERT_XLMR: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing HF_ColBERT_XLMR from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HF_ColBERT_XLMR from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HF_ColBERT_XLMR were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['bert.encoder.layer.10.attention.output.dense.weight', 'bert.encoder.layer.10.intermediate.dense.weight', 'bert.encoder.layer.8.attention.self.key.weight', '

[May 12, 15:42:20] #> Starting from checkpoint /tmp/tmpd8bzu6wg/output_dir/test_training_student/none/2022-05/12/15.40.21/checkpoints/colbert-LAST.dnn
[May 12, 15:42:22] #>>>>> at ColBERT name (model type) : xlm-roberta-base
[May 12, 15:42:22] #>>>>> at BaseColBERT name (model type) : xlm-roberta-base
[May 12, 15:42:22] #> base_config.py load_from_checkpoint xlm-roberta-base
[May 12, 15:42:22] #> base_config.py load_from_checkpoint xlm-roberta-base/artifact.metadata
[May 12, 15:42:22] factory model type: xlm-roberta-base


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing HF_ColBERT_XLMR: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing HF_ColBERT_XLMR from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HF_ColBERT_XLMR from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HF_ColBERT_XLMR were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['bert.encoder.layer.10.attention.output.dense.weight', 'bert.encoder.layer.10.intermediate.dense.weight', 'bert.encoder.layer.8.attention.self.key.weight', '

[May 12, 15:42:32] #> Loading teacher checkpoint /tmp/tmpd8bzu6wg/output_dir/test_training_teacher/none/2022-05/12/15.40.21/checkpoints/colbert-LAST.dnn
[May 12, 15:42:34] distill_query_passage_separately functionality is not supported (yet)
[May 12, 15:42:34] maxsteps: 3
[May 12, 15:42:34] 1 epochs of 5 examples
[May 12, 15:42:34] batch size: 1
[May 12, 15:42:34] maxsteps set to 3
[May 12, 15:42:34] start batch idx: 0
[May 12, 15:42:34] #> XMLR QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
[May 12, 15:42:34] #> Input: $ সে একটা জরুরি পুনর্বাসনের কাজ করছিল, 		 True, 		 None
[May 12, 15:42:34] #> Output IDs: torch.Size([32]), tensor([     0,   9748,  22540,  71896, 237528, 231247,  71994,  31938,  30234,
        168033,   2763,      2,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1])
[May 12, 15:42:34] #> Output Mask: torch.Size([32]), 

/u/franzm/packages/minconda3/envs/ColBERT_primeqa_7/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/u/franzm/packages/minconda3/envs/ColBERT_primeqa_7/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
/u/franzm/packages/minconda3/envs/ColBERT_primeqa_7/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[May 12, 15:42:34] #>>>> before linear query ==
[May 12, 15:42:34] #>>>>> Q: torch.Size([32, 768]), tensor([[-1.5845e-03,  7.9839e-02,  5.0856e-02,  ..., -1.0550e-01,
          9.7534e-02,  1.7295e-02],
        [-8.6329e-03,  2.0454e-02,  4.8183e-05,  ...,  4.7402e-02,
          8.4029e-02,  4.5498e-02],
        [-9.8693e-03,  8.0432e-02, -2.1066e-02,  ..., -1.1526e-01,
          1.0836e-02,  6.8962e-02],
        ...,
        [ 1.9038e-02,  8.9035e-02,  4.3188e-02,  ..., -2.7035e-02,
          5.8574e-02, -2.5945e-02],
        [ 6.7096e-02,  9.8762e-02,  1.1187e-02,  ..., -4.2818e-02,
          5.3879e-02,  4.3672e-02],
        [ 7.1772e-02,  9.2113e-02,  8.6085e-03,  ..., -1.7460e-01,
          6.0348e-02,  4.0607e-03]], grad_fn=<SelectBackward0>)
[May 12, 15:42:34] #>>>>> self.linear query : Parameter containing:
tensor([[-1.2620e-02, -3.6884e-03, -1.0780e-02,  ..., -1.3896e-02,
          1.2237e-02, -8.2247e-03],
        [-3.5069e-03,  3.9557e-03, -2.4896e-02,  ..., -2.6217e-02,
   

#>>>    -3.51 -2.81 		|		 -0.6999999999999997
[May 12, 15:42:42] 0 0.021524619311094284
#>>>    -7.68 -5.89 		|		 -1.79
[May 12, 15:42:50] 1 0.021527044467628
#>>>    -2.86 -4.27 		|		 1.4099999999999997
[May 12, 15:42:57] 2 0.02152507046238333
#> Saving a checkpoint to /tmp/tmpd8bzu6wg/output_dir/test_training/none/2022-05/12/15.40.21/checkpoints/colbert.dnn.batch_3.model ..
#> Saving a checkpoint to /tmp/tmpd8bzu6wg/output_dir/test_training/none/2022-05/12/15.40.21/checkpoints/colbert-batch_3 ..
[May 12, 15:43:04] name:/tmp/tmpd8bzu6wg/output_dir/test_training/none/2022-05/12/15.40.21/checkpoints/colbert-LAST.dnn
[May 12, 15:43:04] Make a sym link of /tmp/tmpd8bzu6wg/output_dir/test_training/none/2022-05/12/15.40.21/checkpoints/colbert.dnn.batch_3.model to /tmp/tmpd8bzu6wg/output_dir/test_training/none/2022-05/12/15.40.21/checkpoints/colbert-LAST.dnn
[May 12, 15:43:04] #> Done with all triples!
#> Saving a checkpoint to /tmp/tmpd8bzu6wg/output_dir/test_training/none/2022-05/12/15.40.

:Next, we run Knowledge Distillation where the student learns teacher's relavance prediction score using parallel data containing:

a) non-English queries and English passages for the student model

b) English queries and English passages for the teacher model.

In [12]:
args_dict = {
    'root': output_dir,
    'experiment': 'test_training',
    'similarity': 'l2',
    'model_type': model_type,
    'checkpoint': stage_1_student_model_fn + '-LAST.dnn',
    'distill_query_passage_separately': False,
    'teacher_model_type': model_type,
    'teacher_checkpoint': teacher_model_fn + '-LAST.dnn',
    'triples': text_triples_en_fn,
    'teacher_triples': text_triples_en_fn,
    'maxsteps': 3,
    'bsize': 1,
    'accumsteps': 1,
    'amp': True,
    'epochs': 1,
    'rank': 0,
    'nranks': 1
}

with Run().context(RunConfig(root=args_dict['root'], experiment=args_dict['experiment'], nranks=args_dict['nranks'], amp=args_dict['amp'])):
    colBERTConfig = ColBERTConfig(**args_dict)
    stage_2_student_model_fn = train(colBERTConfig, text_triples_fn, None, None)


{
    "nprobe": 2,
    "ncandidates": 8192,
    "index_path": null,
    "nbits": 1,
    "kmeans_niters": 20,
    "num_partitions_max": 10000000,
    "similarity": "l2",
    "bsize": 1,
    "accumsteps": 1,
    "lr": 3e-6,
    "maxsteps": 3,
    "save_every": null,
    "resume": false,
    "resume_optimizer": false,
    "warmup": null,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": false,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "shuffle_every_epoch": false,
    "save_steps": 2000,
    "save_epochs": -1,
    "epochs": 1,
    "input_arguments": {},
    "model_type": "xlm-roberta-base",
    "init_from_lm": null,
    "local_models_repository": null,
    "ranks_fn": null,
    "topK": 100,
    "student_teacher_temperature": 1.0,
    "student_teacher_top_loss_weight": 0.5,
    "teacher_model_type": "xlm-roberta-base",
    "teacher_doc_maxlen": 180,
    "distill_query_passage_separately": false,
    "query_only": 

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing HF_ColBERT_XLMR: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing HF_ColBERT_XLMR from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HF_ColBERT_XLMR from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HF_ColBERT_XLMR were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['bert.encoder.layer.10.attention.output.dense.weight', 'bert.encoder.layer.10.intermediate.dense.weight', 'bert.encoder.layer.8.attention.self.key.weight', '

[May 12, 15:43:25] #> Starting from checkpoint /tmp/tmpd8bzu6wg/output_dir/test_training/none/2022-05/12/15.40.21/checkpoints/colbert-LAST.dnn
[May 12, 15:43:26] #>>>>> at ColBERT name (model type) : xlm-roberta-base
[May 12, 15:43:26] #>>>>> at BaseColBERT name (model type) : xlm-roberta-base
[May 12, 15:43:26] #> base_config.py load_from_checkpoint xlm-roberta-base
[May 12, 15:43:26] #> base_config.py load_from_checkpoint xlm-roberta-base/artifact.metadata
[May 12, 15:43:26] factory model type: xlm-roberta-base


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing HF_ColBERT_XLMR: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing HF_ColBERT_XLMR from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HF_ColBERT_XLMR from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HF_ColBERT_XLMR were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['bert.encoder.layer.10.attention.output.dense.weight', 'bert.encoder.layer.10.intermediate.dense.weight', 'bert.encoder.layer.8.attention.self.key.weight', '

[May 12, 15:43:37] #> Loading teacher checkpoint /tmp/tmpd8bzu6wg/output_dir/test_training_teacher/none/2022-05/12/15.40.21/checkpoints/colbert-LAST.dnn
[May 12, 15:43:39] maxsteps: 3
[May 12, 15:43:39] 1 epochs of 5 examples
[May 12, 15:43:39] batch size: 1
[May 12, 15:43:39] maxsteps set to 3
[May 12, 15:43:39] start batch idx: 0
[May 12, 15:43:39] #> XMLR QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
[May 12, 15:43:39] #> Input: $ 중국에서 가장 오랜기간 왕위를 유지한 인물은 누구인가?, 		 True, 		 None
[May 12, 15:43:39] #> Output IDs: torch.Size([32]), tensor([     0,   9748,  24120,   1180,  13968, 211059,  83639,  76826,  78363,
         57104,    993, 161732,    697, 116932, 114150,     32,      2,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1])
[May 12, 15:43:39] #> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 

/u/franzm/packages/minconda3/envs/ColBERT_primeqa_7/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/u/franzm/packages/minconda3/envs/ColBERT_primeqa_7/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
/u/franzm/packages/minconda3/envs/ColBERT_primeqa_7/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[May 12, 15:43:39] #>>>> before linear query ==
[May 12, 15:43:39] #>>>>> Q: torch.Size([32, 768]), tensor([[ 2.6682e-02,  8.1459e-02,  5.4094e-02,  ..., -7.3267e-02,
          5.6192e-02, -1.7213e-04],
        [ 7.2376e-02,  1.0662e-02,  4.2844e-02,  ..., -8.9213e-02,
         -4.9151e-02, -4.5396e-02],
        [ 7.3297e-02,  7.4626e-02, -5.4132e-03,  ...,  4.8789e-02,
          3.8203e-02,  1.5159e-01],
        ...,
        [ 4.9443e-02, -4.9496e-03,  4.1285e-02,  ..., -6.2081e-02,
         -4.3151e-02, -3.4265e-02],
        [ 9.6327e-02,  7.6847e-02,  4.1636e-03,  ..., -4.2079e-02,
         -1.9446e-03,  2.9361e-02],
        [ 1.2414e-01,  6.9005e-02, -3.0700e-02,  ..., -2.4253e-01,
         -4.2438e-02, -3.3461e-02]], grad_fn=<SelectBackward0>)
[May 12, 15:43:39] #>>>>> self.linear query : Parameter containing:
tensor([[-1.2611e-02, -3.6799e-03, -1.0779e-02,  ..., -1.3905e-02,
          1.2242e-02, -8.2162e-03],
        [-3.5072e-03,  3.9577e-03, -2.4895e-02,  ..., -2.6217e-02,
   

[May 12, 15:43:43] #>>>> before linear doc ==
[May 12, 15:43:43] #>>>>> D: torch.Size([155, 768]), tensor([[ 0.1005,  0.0909,  0.0866,  ..., -0.0505,  0.0522, -0.0426],
        [-0.0518,  0.0138,  0.0013,  ..., -0.1404, -0.0883,  0.0109],
        [-0.0147,  0.0238, -0.0356,  ...,  0.0298,  0.0207, -0.0013],
        ...,
        [ 0.0917,  0.0821,  0.0211,  ..., -0.1612, -0.0368,  0.0031],
        [ 0.0917,  0.0821,  0.0211,  ..., -0.1612, -0.0368,  0.0031],
        [ 0.0917,  0.0821,  0.0211,  ..., -0.1612, -0.0368,  0.0031]])
[May 12, 15:43:43] #>>>>> self.linear doc : Parameter containing:
tensor([[-1.2613e-02, -3.6825e-03, -1.0767e-02,  ..., -1.3904e-02,
          1.2236e-02, -8.2245e-03],
        [-3.5072e-03,  3.9648e-03, -2.4897e-02,  ..., -2.6205e-02,
          4.0816e-03, -8.0879e-03],
        [-1.1153e-02, -2.5157e-05,  1.5023e-02,  ..., -1.1317e-02,
         -9.3637e-03, -2.2197e-02],
        ...,
        [-1.0294e-02,  3.2583e-02,  3.3276e-03,  ...,  3.1877e-02,
          3.

## Indexing
Next, we will index a collection of documents, using model representaion from the previous step. 
The collection is a TSV file, containing each document's ID, title, and text.

In [13]:
collection_fn = os.path.join(test_files_location, "xorqa.train_ir_001pct_at_0_pct_collection_fornum.tsv")

Here is an example document:

In [14]:
data = pd.read_csv(collection_fn, sep='\t', header=0, nrows=1)
#data = pd.read_csv(collection_fn, sep='\t', header=None, skiprows=3, nrows=1)
display(HTML(data.to_html()))

,id,text,title
0,1,"The Kangxi Emperor's reign of 61 years makes him the longest-reigning emperor in Chinese history (although his grandson, the Qianlong Emperor, had the longest period of ""de facto"" power) and one of the longest-reigning rulers in the world. However, since he ascended the throne at the age of seven, actual power was held for six years by four regents and his grandmother, the Grand Empress Dowager Xiaozhuang.",Kangxi Emperor


Here are the indexer arguments:

In [15]:
args_dict = {
                'root': os.path.join(output_dir,'test_indexing'),
                'experiment': 'test_indexing',
                'checkpoint': stage_2_student_model_fn,
                'collection': collection_fn,
                'index_root': os.path.join(output_dir, 'test_indexing', 'indexes'),
                'index_name': 'index_name',
                'doc_maxlen': 180,
                'num_partitions_max': 2,
                'kmeans_niters': 1,
                'nway': 1,
                'rank': 0,
                'nranks': 1,
                'amp': True
            }

Here we run the indexer:

In [16]:
with Run().context(RunConfig(root=args_dict['root'], experiment=args_dict['experiment'], nranks=args_dict['nranks'], amp=args_dict['amp'])):
    colBERTConfig = ColBERTConfig(**args_dict)
    create_directory(colBERTConfig.index_path_)
    encode(colBERTConfig, collection_fn, None, None)




[May 12, 15:44:34] #> Creating directory /tmp/tmpd8bzu6wg/output_dir/test_indexing/indexes/index_name 


{
    "nprobe": 2,
    "ncandidates": 8192,
    "index_path": null,
    "nbits": 1,
    "kmeans_niters": 1,
    "num_partitions_max": 2,
    "similarity": "cosine",
    "bsize": 32,
    "accumsteps": 1,
    "lr": 3e-6,
    "maxsteps": 500000,
    "save_every": null,
    "resume": false,
    "resume_optimizer": false,
    "warmup": null,
    "warmup_bert": null,
    "relu": false,
    "nway": 1,
    "use_ib_negatives": false,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "shuffle_every_epoch": false,
    "save_steps": 2000,
    "save_epochs": -1,
    "epochs": 10,
    "input_arguments": {},
    "model_type": "bert-base-uncased",
    "init_from_lm": null,
    "local_models_repository": null,
    "ranks_fn": null,
    "topK": 100,
    "student_teacher_temperature": 1.0,
    "student_teacher_top_loss_weight": 0.5,
    "teacher_model_type": "xlm-

/u/franzm/packages/minconda3/envs/ColBERT_primeqa_7/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
/u/franzm/packages/minconda3/envs/ColBERT_primeqa_7/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[May 12, 15:44:47] #>>>> before linear doc ==
[May 12, 15:44:47] #>>>>> D: torch.Size([180, 768]), tensor([[ 0.0679,  0.0969,  0.0546,  ..., -0.1383,  0.0548,  0.0015],
        [ 0.0008,  0.0251,  0.0328,  ..., -0.1284,  0.0132,  0.1091],
        [-0.0445,  0.0889, -0.0314,  ..., -0.2843, -0.0218,  0.0866],
        ...,
        [-0.0586,  0.0578,  0.0066,  ..., -0.1690, -0.0392, -0.0020],
        [-0.0586,  0.0578,  0.0066,  ..., -0.1690, -0.0392, -0.0020],
        [-0.0586,  0.0578,  0.0066,  ..., -0.1690, -0.0392, -0.0020]])
[May 12, 15:44:47] #>>>>> self.linear doc : Parameter containing:
tensor([[-1.2608e-02, -3.6782e-03, -1.0782e-02,  ..., -1.3897e-02,
          1.2241e-02, -8.2139e-03],
        [-3.5000e-03,  3.9650e-03, -2.4894e-02,  ..., -2.6225e-02,
          4.0926e-03, -8.0725e-03],
        [-1.1159e-02, -1.7327e-05,  1.5027e-02,  ..., -1.1298e-02,
         -9.3554e-03, -2.2198e-02],
        ...,
        [-1.0298e-02,  3.2585e-02,  3.3551e-03,  ...,  3.1883e-02,
          3.

0it [00:00, ?it/s]

[May 12, 15:44:47] [0] 		 #> Encoding 7 passages..
[May 12, 15:44:49] [0] 		 #> Saving chunk 0: 	 7 passages and 1,220 embeddings. From #0 onward.


1it [00:01,  1.67s/it]

[May 12, 15:44:49] offset: 0
[May 12, 15:44:49] chunk codes size(0): 1220
[May 12, 15:44:49] codes size(0): 1220
[May 12, 15:44:49] codes size(): torch.Size([1220])
[May 12, 15:44:49] >>>>partition.size(0): 2
[May 12, 15:44:49] >>>>num_partition: 2
[May 12, 15:44:49] [0] 		 #> Saving the indexing metadata to /tmp/tmpd8bzu6wg/output_dir/test_indexing/indexes/index_name/metadata.json ..


The resulting index files are in `output_dir/test_indexing/indexes/index_name/metadata.json`

## Search
Next, we use the trained model and the index to search the collection, using queries from a TSV query file.

In [17]:
queries_fn = os.path.join(test_files_location, "xorqa.train_ir_001pct_at_0_pct_queries_fornum.tsv")

Here are the search arguments:

In [18]:
args_dict = {
                'root': output_dir,
                'experiment': 'test_indexing' ,
                'checkpoint': stage_2_student_model_fn,
                'model_type': model_type,
                'collection': collection_fn,
                'index_root': output_dir,
                'index_name': 'index_name',
                'queries': queries_fn,
                #'ranks_fn': ranks_fn,
                'bsize': 1,
                'topK': 1,
                'nprobe': 1,
                'nway': 1,
                'rank': 0,
                'nranks': 1,
                'amp': True,
            }

Here we initalize and run the searcher:

In [19]:
with Run().context(RunConfig(root=args_dict['root'], experiment=args_dict['experiment'], nranks=args_dict['nranks'], amp=args_dict['amp'])):
    colBERTConfig = ColBERTConfig(**args_dict)
    searcher = Searcher(args_dict['index_name'], checkpoint=args_dict['checkpoint'], collection=args_dict['collection'], config=colBERTConfig)
    rankings = searcher.search_all(args_dict['queries'], args_dict['topK'])

[May 12, 15:44:50] #> base_config.py from_path /tmp/tmpd8bzu6wg/output_dir/test_indexing/indexes/index_name/metadata.json
[May 12, 15:44:50] #> base_config.py from_path args loaded! 
[May 12, 15:44:50] #> base_config.py from_path args replaced ! 
[May 12, 15:44:50] #> base_config.py load_from_checkpoint /tmp/tmpd8bzu6wg/output_dir/test_training/none/2022-05/12/15.40.21/checkpoints/colbert
[May 12, 15:44:50] #> base_config.py load_from_checkpoint /tmp/tmpd8bzu6wg/output_dir/test_training/none/2022-05/12/15.40.21/checkpoints/colbert/artifact.metadata
[May 12, 15:44:50] #> base_config.py from_path /tmp/tmpd8bzu6wg/output_dir/test_training/none/2022-05/12/15.40.21/checkpoints/colbert/artifact.metadata
[May 12, 15:44:50] #> base_config.py from_path args loaded! 
[May 12, 15:44:50] #> Loading collection...
0M 
[May 12, 15:44:50] #>>>>> at ColBERT name (model type) : /tmp/tmpd8bzu6wg/output_dir/test_training/none/2022-05/12/15.40.21/checkpoints/colbert
[May 12, 15:44:50] #>>>>> at BaseColBERT

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.79it/s]


Here is the search result for our query, containing [query_id, document_id, rank, score]:

In [20]:
rankings.flat_ranking[0]

(-7239279093922981232, 1, 1, 29.890625)

Here is the text of the query record, contataing the ID and text of the query:

In [21]:
with open(queries_fn, 'r') as f:
    for line in f.readlines():
        if str(rankings.flat_ranking[0][0]) == line.split()[0]:
            print(line)

-7239279093922981232	중국에서 가장 오랜기간 왕위를 유지한 인물은 누구인가?



English translation: `Who maintained the throne for the longest time in China?`

Here is the top retrieved document:

In [22]:
with open(collection_fn, 'r') as f:
    for line in f.readlines():
        if str(rankings.flat_ranking[0][1]) == line.split()[0]:
            print(line)

1	"The Kangxi Emperor's reign of 61 years makes him the longest-reigning emperor in Chinese history (although his grandson, the Qianlong Emperor, had the longest period of ""de facto"" power) and one of the longest-reigning rulers in the world. However, since he ascended the throne at the age of seven, actual power was held for six years by four regents and his grandmother, the Grand Empress Dowager Xiaozhuang."	Kangxi Emperor

